<a href="https://csdms.colorado.edu"><img style="float: center; width: 75%" src="https://raw.githubusercontent.com/csdms/project/main/assets/CSDMS-logo-color-tagline-hor.png"></a>

# Functions

In the [diffusion](diffusion.ipynb) and [advection](advection.ipynb) notebooks,
we wrote code
to solve the one-dimensional diffusion and advection equations numerically,
evolve the solutions with time,
and visualize the results.

However, the code in these notebooks is long and complicated and frequently repetitive.
What if we wanted to use the code again,
with different parameters or perhaps even in a different notebook?
Cutting and pasting is tedious, and it can easily lead to errors.

We'd like a way to organize our code so that it's easier to reuse.
Python provides for this by letting us define *functions*.
A function groups code into a program that can be called as a unit.

Before we start,
we'll need Numpy and a NumPy setting for the code in this notebook.

In [ ]:
import numpy as np

np.set_printoptions(precision=1, floatmode="fixed")

## Definition

In the diffusion notebook,
we defined a time step based on a stability criterion
for our numerical solution to the diffusion equation.

Let's group this code into a function.

In [ ]:
def calculate_time_step(grid_spacing, diffusivity):
    return grid_spacing**2 / diffusivity / 4.0

A function definition begins with the keyword `def`,
followed by the name of the function,
followed by a comma-delimited listing of *arguments* (also known as *parameters*) in parentheses,
and ending with a colon `:`.
The code in the body of the function--run when the function is called--must be indented.

We've named our function `calculate_time_step` (naming functions is often an art).
It takes two arguments,
the grid spacing of the model and the diffusivity.
The variables `grid_spacing` and `diffusivity` are *local* to the function--they don't exist outside of the body of the function.
In the body of the function,
the time step is calculated from a stability criterion
and returned to the caller.

## Execution

Call the `calculate_time_step` function with a grid spacing `dx` of $10.0~m$ and a diffusivity `D` of $0.1~m^2 s^{-1}$.

In [ ]:
dx = 10.0
D = 0.1
dt = calculate_time_step(dx, D)

Note that we passed the arguments to the function in the order it expects:
first the grid spacing, then the diffusivity.
Calling a function we define is no different than calling any other Python function.

Print the result.

In [ ]:
print(f"Time step = {dt:.2f} s")

In Python,
we can also pass arguments by name.

In [ ]:
dt1 = calculate_time_step(grid_spacing=dx, diffusivity=D)
dt1 == dt

This feature, *keyword arguments*, makes the function call more readable.

Further,
when passing arguments by name,
we can change the order of the arguments.

In [ ]:
dt2 = calculate_time_step(diffusivity=D, grid_spacing=dx)
dt2 == dt

This makes the function easier to call--you don't have to remember the argument order.

Keyword arguments can be used with any Python function, whether it's made by us or by someone else.

## Additional features

Python functions have many interesting features,
more than we can address here.
We'll focus on a few,
and provide a list of additional resources in the summary. 

### Default arguments

It's often useful to define default values for the arguments in a function.

Let's create another function from a piece of repeated code in the diffusion notebook.
This one sets the initial profile of the diffused quantity
(e.g., temperature, aerosol concentration, sediment, etc.).

In [ ]:
def set_initial_profile(domain_size=100, boundary_left=500, boundary_right=0):
    concentration = np.empty(domain_size)
    concentration[: int(domain_size / 2)] = boundary_left
    concentration[int(domain_size / 2) :] = boundary_right
    return concentration

Note that each of the arguments is assigned a default value.
These are called *default arguments*.
If any argument is omitted from a call to this function,
its default value is used instead.

Call `set_initial_profile` with a domain size `Lx` of $10~m$.

In [ ]:
Lx = 10
C = set_initial_profile(Lx)

Although we omitted the left and right boundary condition values,
the function call didn't raise an error.

Check the result by printing the returned concentration `C`.

In [ ]:
print(C)

The default values for the left and right boundary conditions were applied.

Using default arguments makes calling a function easier.

### Documentation

Let's group one last chunk of repeated code from the diffusion notebook into a function;
in this case, the solver for the one-dimensional diffusion equation.

In [ ]:
def solve1d(concentration, grid_spacing=1.0, time_step=1.0, diffusivity=1.0):
    flux = -diffusivity * np.diff(concentration) / grid_spacing
    concentration[1:-1] -= time_step * np.diff(flux) / grid_spacing

In our new function `solve1d`,
the arguments for the grid spacing, time step, and diffusivity take default values,
but `concentration`, the argument for the diffused quantity, does not.

**Question:** Without looking at the body of the function,
can you tell what variable type the `concentration` argument should be?
A float? A list? A NumPy array?

This is where documentation can help.

The first statement of the body of a function can optionally hold
the function's documentation string, or *docstring*.
It's used to describe the function's purpose, its arguments, and its return value.

Add a docstring to `solve1d`.

In [ ]:
def solve1d(concentration, grid_spacing=1.0, time_step=1.0, diffusivity=1.0):
    """Solve the one-dimensional diffusion equation with fixed boundary conditions.

    Parameters
    ----------
    concentration : ndarray
        The quantity being diffused.
    grid_spacing : float (optional)
        Distance between grid nodes.
    time_step : float (optional)
        Time step.
    diffusivity : float (optional)
        Diffusivity.

    Returns
    -------
    result : ndarray
        The concentration after a time step.

    Examples
    --------
    >>> import numpy as np
    >>> from solver import solve1d
    >>> z = np.zeros(5)
    >>> z[2] = 5
    >>> solve1d(z, diffusivity=0.25)
    array([   0.0,    1.2,    2.5,    1.2,    0.0])
    """
    flux = -diffusivity * np.diff(concentration) / grid_spacing
    concentration[1:-1] -= time_step * np.diff(flux) / grid_spacing

When a function has a docstring,
you can use the `help` function or the questions mark `?` to display it
in a Python session or in a notebook.

In [ ]:
help(solve1d)

In [ ]:
?solve1d

In a notebook,
you can also select the `Shift` + `Tab` keys to view the docstring.

In [ ]:
# Place the cursor in the line below and select the `Shift` + `Tab` keys.
solve1d

Docstring aren't necessary, but they're helpful because they provide information about a function.

Documentation systems such as [Sphinx](https://www.sphinx-doc.org/) use docstrings to produce formatted documentation.
[NumPy](https://numpy.org/doc/1.20/docs/howto_document.html#docstrings) and [Google](https://google.github.io/styleguide/pyguide.html#s3.8.1-comments-in-doc-strings) have style guidelines for docstrings.
It's a good practice to pick a style and use it consistently.

Before we move on, try a simple example of using `solve1d`.
Start by defining a variable, `z`, to diffuse.

In [ ]:
z = np.zeros(5)
z[2] = 5

print(z)

Now call `solve1d` to diffuse `z` for a given time step and diffusivity.

In [ ]:
solve1d(z, diffusivity=0.25, time_step=0.5)

print(z)

## Refactoring the diffusion example

Let's combine the functions we've defined above into a new function that replicates the diffusion example.

In [ ]:
def diffusion_example():
    """An example of using `solve1d` in a diffusion problem."""
    print(diffusion_example.__doc__)
    D = 100  # diffusivity
    Lx = 10  # domain length
    dx = 0.5  # grid spacing

    dt = calculate_time_step(dx, D)
    C = set_initial_profile(Lx)

    print("Time = 0\n", C)
    for t in range(1, 5):
        solve1d(C, dx, dt, D)
        print(f"Time = {t*dt:.4f}\n", C)

This is a first taste of how programs can be built to solve a problem:
break the problem into smaller pieces,
write functions to address the smaller pieces,
then assemble the functions to solve the problem.

Run the example `diffusion_example`.

In [ ]:
diffusion_example()

## Exercises

1.  "Adding" two strings produces their concatenation: `'a' + 'b'` is `'ab'`. Write a function called `fence` that takes two parameters, `original` and `wrapper`, and returns a new string that has the wrapper character at the beginning and end of the original.

1. Write a function, `normalize`, that takes an array as input and returns a corresponding array of values scaled to the range $[0,1]$. (Hint: Look at NumPy functions such as `arange` and `linspace` to see how their arguments are defined.)

1.  Rewrite your `normalize` function so that it scales data to $[0,1]$ by default, but allows a user to optionally specify the lower and upper bounds.

## Summary

The process of building larger programs from smaller functions is a key element of scientific programming.

Information from the Python documentation, including the sections
[Defining Functions](https://docs.python.org/3/tutorial/controlflow.html#defining-functions) and
[More on Defining Functions](https://docs.python.org/3/tutorial/controlflow.html#more-on-defining-functions)
was used heavily in creating this notebook.
There's a lot more there, including many features of functions we didn't cover.

### Topics not covered

These are a few topics that we didn't cover in this lesson,
but they're important enough that we probably should have.

* *formal* versus *actual* parameters
* the concept of *scope*
* *local* versus *global* variables
* use of *type hints*

More information on these topics can be found in the Python documentation.

### Last thoughts

If your function doesn't fit on a screen, it's too long.
Break it up into smaller functions.

How do we know a function is working as expected?
This is partially answered with *unit testing*, covered later.